# Here emotion prediction was done by using Bi directional LSTM

In [ ]:
import os
import fnmatch
import numpy as np

from sys     import argv
#from sklearn import svm

from load_features     import load_all
#from calc_scores       import calc_scores
#from write_predictions import write_predictions

# Set folders here
path_test_predictions = "D:/AVEC_17_Emotion_Sub-Challenge/result/"
b_test_available      = False  # If the test labels are not available, the predictions on test are written into the folder 'path_test_predictions'

# Folders wi o_features = "E:/AVEC_17_Emotion_Sub-Challenge/audio_features_xbow_6s/"
path_audio_features = "D:/AVEC_17_Emotion_Sub-Challenge/audio_features_xbow_6s/"
path_video_features = "D:/AVEC_17_Emotion_Sub-Challenge/video_features_xbow_6s/"
path_text_features  = "D:/AVEC_17_Emotion_Sub-Challenge/text_features_xbow_6s/"
path_labels         = "D:/AVEC_17_Emotion_Sub-Challenge/labels/"

sr_labels = 0.1

delay = 0.00
b_audio = True
b_video = True
b_text  = True
rise = 0.20





path_features = []
if b_audio:
    path_features.append( path_audio_features )
if b_video:
    path_features.append( path_video_features )
if b_text:
    path_features.append( path_text_features )
        

if not b_test_available and not os.path.exists(path_test_predictions):
    os.mkdir(path_test_predictions)

# Compensate the delay (quick solution)
shift = int(np.round(delay/sr_labels))
shift = np.ones(len(path_features),dtype=int)*shift



files_train = fnmatch.filter(os.listdir(path_features[0]), "Train*")  # Filenames are the same for audio, video, text & labels
files_devel = fnmatch.filter(os.listdir(path_features[0]), "Devel*")
files_test  = fnmatch.filter(os.listdir(path_features[0]), "Test*")



    

# Load features and labels
Train   = load_all( files_train, path_features, shift )
Devel   = load_all( files_devel, path_features, shift )
Train_L = load_all( files_train, [ path_labels ] )  # Labels are not shifted
Devel_L = load_all( files_devel, [ path_labels ] )

# Making windowing for train data.Like input sequence will be[t1,t2,t3],[t2,t3,t4]


In [ ]:
    def create_data(dataset,look_back=1):
        dataX = []
        for i in range(len(dataset)-look_back+1):
            a = dataset[i:i+look_back,:]
            dataX.append(a)
        return np.array(dataX)

    def create_dataY(dataset,look_back=1):
        dataX = []
        for i in range(len(dataset)-look_back+1):
            a = dataset[i:i+look_back,]
            dataX.append(a)
        return np.array(dataX)

 

In [ ]:
   
    
    Train_LA = Train_L[:,0]
    TrainX = Train
    TrainY = Train_LA

    look_back = 6
    
    New_TrainX = create_data(TrainX,look_back)
    New_TrainY = create_dataY(TrainY,look_back)
#TrainY[1:3,]

    Devel_X = create_data(Devel,look_back)
    Devel_Y = create_dataY(Devel_L[:,0],look_back)
    
    
    Y = New_TrainY.reshape(len(TrainX)-look_back+1,look_back,1)
    X = New_TrainX.reshape(len(TrainX)-look_back+1,look_back,4521)
#Devel_X.shape
    Devel_Y = Devel_Y.reshape(len(Devel)-look_back+1,look_back,1)
    
    
   

# training model

In [ ]:
    np.random.seed(5)
    CCC_max = list()
    PCC_max = list()
    for a in range(12):
        import tensorflow as tf
        from keras.models import Sequential
        from keras.layers import LSTM
        from keras.layers import Dense
        from keras.layers import TimeDistributed
        from keras.layers import Bidirectional

        model = Sequential()
        model.add(Bidirectional(LSTM(200,return_sequences = True),input_shape=(look_back,4521)))
        model.add(Bidirectional(LSTM(200,return_sequences = True),input_shape=(look_back,4521)))
        model.add(Bidirectional(LSTM(100,return_sequences = True),input_shape=(look_back,4521)))
        model.add(Bidirectional(LSTM(100,return_sequences = True),input_shape=(look_back,4521)))

        model.add(TimeDistributed(Dense(1, activation= 'linear' )))
        model.compile(loss = 'mean_squared_error',optimizer = 'adam',metrics = ['accuracy'],options = run_opts)
        print(model.summary())
    
    

In [ ]:
        #Devel_X = Devel.reshape(22727,1,4521)
        #Devel_Y = Devel_L[:,0].reshape(22727,1,1)

        model.fit(X,Y,epochs = 1,validation_data=(Devel_X, Devel_Y))
        pred = model.predict(Devel_X)
        

# Calculating CCC,PCC,RMSE

In [ ]:
        
        a = []
        for i in range(len(pred)):
            a.append(pred[i,0])
        for j in range(1,look_back):
            a.append(pred[i,j])
        a = np.array(a)

        a = a.reshape(22727,)
    
    
        #pred = pred.reshape(22727,)

        x = a
        y = Devel_L[:,0]
#y = Y
        x_mean = np.nanmean(x)
        y_mean = np.nanmean(y)

#np.nanmean((x - x_mean) * (y - y_mean)) 
        covariance = np.nanmean((x-x_mean)*(y-y_mean))
    
        x_var = 1.0 / (len(x)-1) * np.nansum((x-x_mean)**2) # Make it consistent with Matlab's nanvar (division by len(x)-1, not len(x)))
        y_var = 1.0 / (len(y)-1) * np.nansum((y-y_mean)**2)
    
        CCC = (2*covariance) / (x_var + y_var + (x_mean-y_mean)**2)

        x_std = np.sqrt(x_var)
        y_std = np.sqrt(y_var)
        PCC = covariance / (x_std * y_std)
    
        RMSE = np.sqrt(np.nanmean((x - y)**2))
    
        CCC_max.append(CCC)
        PCC_max.append(PCC)
        model.reset_states()
        print(CCC_max)
    CCC_W = max(CCC_max)
    PCC_W = max(PCC_max)
    
    

# Writing to a file 

In [2]:
    with open("results_ccc_timestep_6.txt", 'a') as myfile:
    #myfile.write(" Arousal Valence Liking\n")
        myfile.write(str(CCC_W) + '\n')
    with open("results_pcc_timestep_6.txt", 'a') as myfile:
    #myfile.write("Arousal Valence Liking\n")
        myfile.write(str(PCC_W) + '\n')
    model.reset_states()
    delay = delay + rise
    print(delay)

Inside run_baseline.py
3
argv
['c:\\users\\abhijit\\anaconda3\\envs\\tensorflow-gpu\\lib\\site-packages\\ipykernel_launcher.py', '-f', 'C:\\Users\\abhijit\\AppData\\Roaming\\jupyter\\runtime\\kernel-a3b15971-281f-4f10-96f3-a4db35a275aa.json']
delay
0.0
i am going
['Train_01.csv', 'Train_02.csv', 'Train_03.csv', 'Train_04.csv', 'Train_05.csv', 'Train_06.csv', 'Train_07.csv', 'Train_08.csv', 'Train_09.csv', 'Train_10.csv', 'Train_11.csv', 'Train_12.csv', 'Train_13.csv', 'Train_14.csv', 'Train_15.csv', 'Train_16.csv', 'Train_17.csv', 'Train_18.csv', 'Train_19.csv', 'Train_20.csv', 'Train_21.csv', 'Train_22.csv', 'Train_23.csv', 'Train_24.csv', 'Train_25.csv', 'Train_26.csv', 'Train_27.csv', 'Train_28.csv', 'Train_29.csv', 'Train_30.csv', 'Train_31.csv', 'Train_32.csv', 'Train_33.csv', 'Train_34.csv']
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_5 (Bidirection (None, 6, 400)            7555

KeyboardInterrupt: 